In [1]:
from importlib.metadata import version
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import numpy as np
import math
from dataclasses import dataclass
from matplotlib import pyplot as plt
import time
import os

# Pour torch si vous avez un GPU
# device = "cpu" if not torch.cuda.is_available() else "cuda"
device = "cpu" # Pour forcer l'utilisation du CPU

In [2]:
# Environement potentielement testé
from environnement.environnement import Environnement as env # mother class
from environnement.environnement1 import Environnement1 as env1
from environnement.environnement2Str import Environnement2 as env2Str
from environnement.environnement3Str import Environnement3 as env3Str
from environnement.environnement6Str import Environnement6 as env6Str
from environnement.small_loop import small_loop
from environnement.small_loop import WorldDisplay
from ipywidgets import Output

# model machine learning
from model.DeepNN import *
from model.Tokenizer import *
from model.CustomDataSet import CustomDataSet
from outil import *
from inter.interactions import Interaction
from inter.simpleInteraction import simpleInteraction as inter

# L'agent qui devine les meilleurs patterns
Je vais maintenant implémenter un mécanisme qui va tenter de trouver les meilleurs pattern.
## Critere d'un pattern :
Un bon pattern est un pattern pour lequel Prob * Valance est élever. Nous voulons que l'agent sout capable de dire : cette action est bonne car après je vais pouvoir avoir une bonne valence. Donc il faut prédire sur du long terme.
## L'idée
L'idée est de regarder les probabilités que donne le modèl pour certainne action. Créer des enchainements d'actions, en découlera des prédictions de pattern. Nous pourrons alors éxecuter des bonnes séquence d'action.

## Valence
La valence est primordiale pour notre. C'est elle qui va faire emerger un comportement à notre agent. 

### Exemple small loop
Prennons un environement un peu complexe. On peut imaginer un monde en 2 dimensions, sur une grille de 5x5 avec les bords et la case du centre comme obstacle. Nous voulons que notre agent apprenne a ce balader sans se prendre d'obstacle. Graphiquement cela donne :

In [3]:
demo_env = small_loop(x=1, y=1, theta=0)
# demo_env.display_world()

Imaginons que nous avons un modèle prédictif parfait. Il faut choisir la meilleur action. Pour simplifier disons que nous pouvons prédire que sur 3 interactions futur, sans connaitre la position absolut du robot la meilleur solution est la suivante :
- **Feel front** regarder si devans nous il y a un obstacle. Nous voulons que la prédiction soit égal à 50% wall 50% Empty. En tout cas une probabilité qui s'en rapproche.
- **move** Nous voulons que le robot agisse en fonction, soit avancé soit touner.

Si nous somme dans une intéraction ou nous avons `feel front` alors nous voulons directement faire l'action qui correspond.

In [4]:
class AgentPE:
    def __init__(self, model, all_outcomes, all_actions, tokenizer, optimizer=None, loss_func=None):
        """ 
        Création de l'agent.
        
        - self._action : action précédente
        - self._predicted_outcome : prédiction de l'outcome précédent
        """
        self._action = None
        self._predicted_outcome = None
        self._model = model
        self._otimizer = optimizer
        self._loss_func = loss_func
        self._tokenizer:SimpleTokenizerV1 = tokenizer
        self._all_outcomes = all_outcomes
        self._all_actions = all_actions
        self._history_act = []
        self._history_fb = []


    def fit(self, actions:list, outcomes:list, nb_epoch:int= 5, validate_loader=None):
        """
        Fonction d'entrainement de l'agent 
        Avec data set custom, le model prends en inputs plusieurs données
        """
        context_lenght = self._model.input_size
        if len(actions) + len(outcomes) < context_lenght:
            raise Exception("Not enough data to train model")

        actions = [self._tokenizer.encode(act) for act in actions]
        outcomes = self._tokenizer.encode(outcomes)
        
        if isinstance(self._model, torch.nn.Module):
            self._model.train()
            # actions = torch.tensor(actions, dtype=torch.float).to(device) # On passe toutes les actions que l'agent a fais
            # outcomes = torch.tensor(outcomes, dtype=torch.long).to(device) # On passe toutes oputcomes qu'il a 

            data_loarder = CustomDataSet(actions=actions, outcomes=outcomes, # On va créer un dataset
                         context_lenght=context_lenght, dim_out=len(self._all_outcomes))
            
            # A la place de donnée x = [act1] y = [out1]
            # Nous voulons donné : x = [act1, out1, act2] y = [out2]

            data_loader = torch.utils.data.DataLoader( # On utilise torch pour charger les données
                data_loarder,batch_size=32, shuffle=True)

            train_with_batch(model=self._model, 
                    train_loader=data_loader,
                    optimizer=self._otimizer,
                    loss_func=self._loss_func,
                    nb_epochs=nb_epoch,
                    validate_loader=validate_loader,
                    print_=True)
        else: # Si le model n'est pas un model pytorch
            raise Exception('Not implemented')
            self._model.fit(action, outcome)
            pass

    def get_prediction(self, action):
        gap = (self._model.input_size - 1) // 2
        x = []
        for i in range(len(self._history_act) - gap, len(self._history_act)):
            x.append(self._history_act[i])
            x.append(self._history_fb[i])
        x.append(action)
        action = self._tokenizer.encode(x)
        
        if isinstance(self._model, torch.nn.Module):
            self._model.eval() 
            action = torch.tensor(action, dtype=torch.float).to(device)
            x = self._model(action)
            x = torch.nn.functional.softmax(x, dim=0)
        else:
            raise Exception('Not implemented')
            x=self._model.predict(action)
        
        return x
       # def check_all_actions(self):
    #     act_to_test = None
    #     for act in self._all_actions:
    #         if act not in self._history_act:
    #             act_to_test = act
    #             break
    #     return act_to_test
    
    # def decide(self):
    #     """
    #     Fonction qui choisit l'action a faire en fonction des prédictions \
    #     du modèles entrainné. Nous renforçons choisisons les actions que \
    #     ou le modèle n'est pas sûr.
    #     """

    #     act_test = self.check_all_actions()
    #     if act_test:
    #         print("i don't know", act_test)
    #         self._action = act_test
    #         return act_test

    #     best_act = self._all_actions[0]
    #     best_expected_val = -np.inf

    #     # On vérifie que l'on a vue assez d'interaction pour faire des prédictions
    #     if len(self._history_act) + len(self._history_fb) < self._model.input_size:
    #         print("Not enough data to make a decision")
    #         return best_act

    #     # Vérifie si le modèles est sur de sa prédiction
    #     for act in self._all_actions:
    #         probs:torch.Tensor = self.get_prediction(act)
    #         max_prob = torch.max(probs).item()
    #         # Formule utiliser : 1 / n + 0.5 / n
    #         print(f'for action {act} probs {probs} max_prob {max_prob}')
    #         if max_prob < 1 / probs.size(dim=0) + 0.5 / probs.size(dim=0):
    #             print("je ne suis pas sur de ", act)
    #             if len(self._all_actions) + len(self._all_outcomes) > self._model.input_size:
    #                 self.fit(self._history_act, self._history_fb, validate_loader=None)

    #         probs:torch.Tensor = self.get_prediction(act)
    #         if max_prob < 1 / probs.size(dim=0) + 0.5 / probs.size(dim=0):
    #             print("je n'arrive pas à être sur de ", act)
    #             return act
            
    #         # Si le modèle as une prédiction sur, on regarde sa valance
    #         predi = self._tokenizer.decode(torch.argmax(probs, dim=0).item())
    #         expected_val = self._valance[inter(act, predi)]
    #         if expected_val > best_expected_val:
    #             best_act = act
    #             best_expected_val = expected_val
    #             print(f"Action: {act}, Expected valance: {expected_val}")
    #     self._action = best_act
    #     return best_act

    # Modifier
    def predict(self, action):
        """
        Funciton de prédiction
        """

        if len(self._history_act) + len(self._history_fb) + 1 < self._model.input_size:
            raise Exception("Not enough data to train model")
        
        gap = (self._model.input_size - 1) // 2
        x = []
        for i in range(len(self._history_act) - gap, len(self._history_act)):
            x.append(self._history_act[i])
            x.append(self._history_fb[i])
        x.append(action)
        action = self._tokenizer.encode(x)
        if isinstance(self._model, torch.nn.Module):
            self._model.eval() 
            action = torch.tensor(action, dtype=torch.float).to(device)
            x = self._model(action)
            x = torch.argmax(x, dim=0).item()

        else:
            raise Exception('Not implemented')
            x=self._model.predict(action)
        
        return self._tokenizer.decode(x)

    # Modofier
    def action(self, outcome, fit=True, validate_loader=None):
        """ 
        Fonction qui choisit l'action a faire en fonction de la dernière \
        intéraction avec l'environnement. \n
        C'est ici que nous allons implémenter un mécanisme de ML \
        pour choisir la prochaine action.

        :param: **outcome** feedback de la dernière intéraction avec l'environnement

        :return: **action** action à effectuer
        """
        if self._action is not None:
            self._history_fb.append(outcome)
            print(f"Action: {self._action}, Prediction: {self._predicted_outcome}, Outcome: {outcome}, " 
                  f"\033[0;31m Satisfaction: {self._predicted_outcome == outcome} \033[0m")
            if self._predicted_outcome != outcome:
                if len(self._history_act) + len(self._history_fb) > self._model.input_size:
                    self.fit(self._history_act, self._history_fb, validate_loader=validate_loader)
                else:
                    self._action = self._all_actions[0]

            # Maintenant nous choisissons la prochaine action en fonction de la valance
            self._action = str(np.random.choice(self._all_actions))
            if len(self._history_act) + len(self._history_fb) + 1 > self._model.input_size:
                self._predicted_outcome = self.predict(self._action)
            self._history_act.append(self._action)
        else:
            self._action = self._all_actions[0]
            self._history_act.append(self._action)            
            print(f"Action de base : {self._action} Prediction: {self._predicted_outcome}")
        
        return self._action, self._predicted_outcome

In [5]:
env_test2 = small_loop(x=1, y=1, theta=0)

model_ML = DeepNetwork(hidden_size=[16], input_size=3, output_size=2)
optimizer = torch.optim.Adam(model_ML.parameters(), lr=0.01, weight_decay=0.001)
loss_func = nn.CrossEntropyLoss()
tokenizer = SimpleTokenizerV1(create_dico_numerate_word(env_test2.get_outcomes() + env_test2.get_actions()))

agent_test2 = AgentPE(
    model=model_ML,
    all_outcomes= env_test2.get_outcomes(),
    all_actions= env_test2.get_actions(),
    tokenizer=tokenizer,
    optimizer=optimizer,
    loss_func=loss_func)

history_good = []
pourcent_by_10 = []
outcome = None
for i in range(200):
    print(f"=======================\033[0;32m iteration {i} \033[0m=======================")
    action, predi = agent_test2.action(outcome, False)
    outcome = env_test2.outcome(action)
    history_good.append(outcome == predi)
    pourcent_by_10.append(sum(history_good[-10:]) * 10 if len(history_good) >= 10 else 0)
    print(f'action {action} predi {predi} outcome {outcome}')
    print(f"Action choisie : {action} \033[0;34m{pourcent_by_10[-1]} \033[0m")
    print("\n")
    

liste hidden init [16]
======================= iteration 0 =======================
Action de base : forward Prediction: None
action forward predi None outcome wall
Action choisie : forward 0 


======================= iteration 1 =======================
Action: forward, Prediction: None, Outcome: wall,  Satisfaction: False 
action turn_right predi None outcome empty
Action choisie : turn_right 0 


======================= iteration 2 =======================
Action: turn_right, Prediction: None, Outcome: empty,  Satisfaction: False 
Epoch 1/5, Loss: 0.4942
Epoch 2/5, Loss: 0.4242
Epoch 3/5, Loss: 0.3585
Epoch 4/5, Loss: 0.2994
Epoch 5/5, Loss: 0.2474
action turn_left predi empty outcome empty
Action choisie : turn_left 0 


======================= iteration 3 =======================
Action: turn_left, Prediction: empty, Outcome: empty,  Satisfaction: True 
action turn_left predi empty outcome empty
Action choisie : turn_left 0 


======================= iteration 4 =====================

/home/pe/Documents/stage/Dpt_transformers/.venv/lib/python3.11/site-packages/torch/autograd/graph.py:823: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 5/5, Loss: 0.5828
action turn_left predi wall outcome empty
Action choisie : turn_left 60 


======================= iteration 85 =======================
Action: turn_left, Prediction: wall, Outcome: empty,  Satisfaction: False 
Epoch 1/5, Loss: 0.5642
Epoch 2/5, Loss: 0.9175
Epoch 3/5, Loss: 0.4842
Epoch 4/5, Loss: 0.2594
Epoch 5/5, Loss: 0.6086
action forward predi empty outcome empty
Action choisie : forward 60 


======================= iteration 86 =======================
Action: forward, Prediction: empty, Outcome: empty,  Satisfaction: True 
action turn_left predi empty outcome empty
Action choisie : turn_left 60 


======================= iteration 87 =======================
Action: turn_left, Prediction: empty, Outcome: empty,  Satisfaction: True 
action turn_left predi empty outcome empty
Action choisie : turn_left 60 


======================= iteration 88 =======================
Action: turn_left, Prediction: empty, Outcome: empty,  Satisfaction: True 
action forward 

In [6]:
# On teste certaines sequences de prédiction
print("Test de prédiction")
seq = tokenizer.encode(['forward', 'wall', 'forward'])
seq = torch.tensor(seq, dtype=torch.float).to(device)
predi =  agent_test2._model(seq)
prob = torch.nn.functional.softmax(predi, dim=0)
deocde = tokenizer.decode(torch.argmax(predi, dim=0).item())
print(f"Prédiction de la séquence 'forward', 'wall', 'forward', probabilité {prob}, decode {deocde}")

seq = tokenizer.encode(['feel_front', 'empty', 'forward'])
seq = torch.tensor(seq, dtype=torch.float).to(device)
predi =  agent_test2._model(seq)
prob = torch.nn.functional.softmax(predi, dim=0)
print(prob)
deocde = tokenizer.decode(torch.argmax(predi, dim=0).item())
print(f"Prédiction de la séquence 'feel_front', 'empty', 'forward' probabilité {prob} decode {deocde}")

seq = tokenizer.encode(['feel_front', 'wall', 'forward'])
seq = torch.tensor(seq, dtype=torch.float).to(device)
predi =  agent_test2._model(seq)
prob = torch.nn.functional.softmax(predi, dim=0)
deocde = tokenizer.decode(torch.argmax(predi, dim=0).item())
print(f"Prédiction de la séquence 'feel_front', 'wall', 'forward', probabilité {prob} decode {deocde}")

# Cas non prédictible correctement :
seq = tokenizer.encode(['forward', 'empty', 'forward'])
seq = torch.tensor(seq, dtype=torch.float).to(device)
predi =  agent_test2._model(seq)
prob = torch.nn.functional.softmax(predi, dim=0)
deocde = tokenizer.decode(torch.argmax(predi, dim=0).item())
print(f"Prédiction de la séquence forward', 'empty', 'forward' :  probabilité {prob}, decode {deocde}")

seq = tokenizer.encode(['turn_left', 'empty', 'forward'])
seq = torch.tensor(seq, dtype=torch.float).to(device)
predi =  agent_test2._model(seq)
prob = torch.nn.functional.softmax(predi, dim=0)
deocde = tokenizer.decode(torch.argmax(predi, dim=0).item())
print(f"Prédiction de la séquence 'turn_left', 'empty', 'forward'  probabilité {prob}, decode {deocde}")

seq = tokenizer.encode(['turn_right', 'empty', 'forward'])
seq = torch.tensor(seq, dtype=torch.float).to(device)
predi =  agent_test2._model(seq)
prob = torch.nn.functional.softmax(predi, dim=0)
deocde = tokenizer.decode(torch.argmax(predi, dim=0).item())
print(f"Prédiction de la séquence 'turn_right', 'empty', 'forward', probabilité {prob}, decode {deocde}")

Test de prédiction
Prédiction de la séquence 'forward', 'wall', 'forward', probabilité tensor([0.5448, 0.4552], grad_fn=<SoftmaxBackward0>), decode wall
tensor([0.2206, 0.7794], grad_fn=<SoftmaxBackward0>)
Prédiction de la séquence 'feel_front', 'empty', 'forward' probabilité tensor([0.2206, 0.7794], grad_fn=<SoftmaxBackward0>) decode empty
Prédiction de la séquence 'feel_front', 'wall', 'forward', probabilité tensor([0.6026, 0.3974], grad_fn=<SoftmaxBackward0>) decode wall
Prédiction de la séquence forward', 'empty', 'forward' :  probabilité tensor([0.2412, 0.7588], grad_fn=<SoftmaxBackward0>), decode empty
Prédiction de la séquence 'turn_left', 'empty', 'forward'  probabilité tensor([0.2342, 0.7658], grad_fn=<SoftmaxBackward0>), decode empty
Prédiction de la séquence 'turn_right', 'empty', 'forward', probabilité tensor([0.2273, 0.7727], grad_fn=<SoftmaxBackward0>), decode empty


In [7]:
# 1er version de la fonction qui détermine l'expected valence :

def tempo_recursif_expective_valance(model:nn.Module, env, seq:list,
                                    max_depth:int, valance:dict, tokenizer:SimpleTokenizerV1,
                                    seuil:float=0.2, proba:float = 1,
                                    seq_predi:list = []):
    """
    """
    if max_depth == 0:
        return {}
    max_depth -= 1

    if proba < seuil:
        return {}
    
    model.eval()
    exceptive_valance = {}
    for act in env.get_actions():
        new_seq = seq_predi + [act]
        seq_to_predict = seq + [tokenizer.encode(act)]
        seq_to_predict = torch.tensor(seq_to_predict, dtype=torch.float).to(device)
        x = model(seq_to_predict)
        # Transforme x into list proba
        probs = torch.nn.functional.softmax(x, dim=0).tolist()
        # for each outcomes we want proba with act
        for i, out in enumerate(env.get_outcomes()):
            tmp_new_seq = new_seq + [out]
            tmp_proba = probs[i] * proba
            if tmp_proba < seuil:
                continue
            tempo = float(np.round(valance[inter(act, out)] * tmp_proba, decimals=4))
            # input(f'seq {seq_predi} act {act} out {out} proba {tmp_proba} valance {valance[(act, out)]} tempo {tempo}')

            exceptive_valance.update(
                tempo_recursif_expective_valance(model=model, env=env, seq=seq[2:] + [tokenizer.encode(act), tokenizer.encode(out)],
                                            max_depth=max_depth, valance=valance, seuil=seuil, 
                                            proba=tmp_proba, seq_predi=tmp_new_seq.copy(), tokenizer=tokenizer)
            )
            exceptive_valance[str(tmp_new_seq)] = tempo
    return exceptive_valance


valence = {
    inter('forward', 'empty') : 10,
    inter('forward', 'wall') : -30,
    inter('turn_left', 'empty') : -3,
    inter('turn_left', 'wall') : -3,
    inter('turn_right', 'empty') : -3,
    inter('turn_right', 'wall') : -3,
    inter('feel_front', 'wall') : -5,
    inter('feel_front', 'empty') : -5,
}

print(env_test2.get_actions())
print(env_test2.get_outcomes())


res = tempo_recursif_expective_valance(model=model_ML, env=env_test2,
        seq=tokenizer.encode(['feel_front', 'wall']), max_depth=3, valance=valence, tokenizer=tokenizer, seuil=0.2)

print(res)

# Top 5 of sequences with the best expected valance
top_5 = sorted(res.items(), key=lambda x: x[1], reverse=True)[:5]
for top in top_5:
    print(f"Sequence: {top[0]} Expected valance: {top[1]}")


['forward', 'turn_left', 'turn_right', 'feel_front']
['wall', 'empty']
{"['forward', 'wall', 'forward', 'wall', 'turn_left', 'empty']": -0.7364, "['forward', 'wall', 'forward', 'wall', 'turn_right', 'empty']": -0.6432, "['forward', 'wall', 'forward', 'wall', 'feel_front', 'wall']": -1.196, "['forward', 'wall', 'forward', 'wall']": -9.8501, "['forward', 'wall', 'forward', 'empty', 'forward', 'empty']": 2.0815, "['forward', 'wall', 'forward', 'empty', 'turn_left', 'empty']": -0.7346, "['forward', 'wall', 'forward', 'empty', 'turn_right', 'empty']": -0.7139, "['forward', 'wall', 'forward', 'empty']": 2.7431, "['forward', 'wall', 'turn_left', 'empty', 'forward', 'empty']": 3.4501, "['forward', 'wall', 'turn_left', 'empty', 'turn_left', 'empty']": -1.2114, "['forward', 'wall', 'turn_left', 'empty', 'turn_right', 'empty']": -1.2093, "['forward', 'wall', 'turn_left', 'empty', 'feel_front', 'empty']": -1.3959, "['forward', 'wall', 'turn_left', 'empty']": -1.3516, "['forward', 'wall', 'turn_rig

In [8]:
print("probalité")
seq = tokenizer.encode(['forward', 'wall', 'forward'])
seq = torch.tensor(seq, dtype=torch.float).to(device)
predi =  agent_test2._model(seq)
prob = torch.nn.functional.softmax(predi, dim=0)
deocde = tokenizer.decode(torch.argmax(predi, dim=0).item())
print(f"Prédiction de la séquence 'turn_right', 'empty', 'forward', probabilité {prob}, decode {deocde}")

probalité
Prédiction de la séquence 'turn_right', 'empty', 'forward', probabilité tensor([0.5448, 0.4552], grad_fn=<SoftmaxBackward0>), decode wall


In [9]:
class AgentPEBis:
    def __init__(self, model, all_outcomes, all_actions, tokenizer, valence:dict,
                optimizer=None, loss_func=None):
        """ 
        Création de l'agent.
        
        - self._action : action précédente
        - self._predicted_outcome : prédiction de l'outcome précédent
        """
        self._action = None
        self._predicted_outcome = None
        self._model = model
        self._otimizer = optimizer
        self._loss_func = loss_func
        self._tokenizer:SimpleTokenizerV1 = tokenizer
        self._all_outcomes = all_outcomes
        self._all_actions = all_actions
        self._history_act = []
        self._history_fb = []
        self._valence = valence


    def fit(self, actions:list, outcomes:list, nb_epoch:int= 20, validate_loader=None):
        """
        Fonction d'entrainement de l'agent 
        Avec data set custom, le model prends en inputs plusieurs données
        """
        context_lenght = self._model.input_size
        if len(actions) + len(outcomes) < context_lenght:
            raise Exception("Not enough data to train model")

        actions = [self._tokenizer.encode(act) for act in actions]
        outcomes = self._tokenizer.encode(outcomes)
        
        if isinstance(self._model, torch.nn.Module):
            self._model.train()
            data_loarder = CustomDataSet(actions=actions, outcomes=outcomes,
                         context_lenght=context_lenght, dim_out=len(self._all_outcomes))

            data_loader = torch.utils.data.DataLoader(
                data_loarder,batch_size=32, shuffle=True)

            train_with_batch(model=self._model, 
                    train_loader=data_loader,
                    optimizer=self._otimizer,
                    loss_func=self._loss_func,
                    nb_epochs=nb_epoch,
                    validate_loader=validate_loader,
                    print_=True)
        else: # Si le model n'est pas un model pytorch
            raise Exception('Not implemented')
            self._model.fit(action, outcome)
            pass

    def get_prediction(self, action):
        gap = (self._model.input_size - 1) // 2
        x = []
        for i in range(len(self._history_act) - gap, len(self._history_act)):
            x.append(self._history_act[i])
            x.append(self._history_fb[i])
        x.append(action)
        action = self._tokenizer.encode(x)
        
        if isinstance(self._model, torch.nn.Module):
            self._model.eval() 
            action = torch.tensor(action, dtype=torch.float).to(device)
            x = self._model(action)
            x = torch.nn.functional.softmax(x, dim=0)
        else:
            raise Exception('Not implemented')
            x=self._model.predict(action)
        
        return x

    def predict(self, action):
        """
        Funciton de prédiction
        """

        if len(self._history_act) + len(self._history_fb) + 1 < self._model.input_size:
            raise Exception("Not enough data to train model")
        
        gap = (self._model.input_size - 1) // 2
        x = []
        for i in range(len(self._history_act) - gap, len(self._history_act)):
            x.append(self._history_act[i])
            x.append(self._history_fb[i])
        x.append(action)
        action = self._tokenizer.encode(x)
        if isinstance(self._model, torch.nn.Module):
            self._model.eval() 
            action = torch.tensor(action, dtype=torch.float).to(device)
            x = self._model(action)
            x = torch.argmax(x, dim=0).item()

        else:
            raise Exception('Not implemented')
            x=self._model.predict(action)
        
        return self._tokenizer.decode(x)
    
    def recursif_expective_valance(self, seq:list, max_depth:int, seuil:float=0.2, proba:float = 1,
                                    seq_predi:list = []):
        """
        """
        if max_depth == 0:
            return {}
        max_depth -= 1

        if proba < seuil:
            return {}
        
        self._model.eval()
        exceptive_valance = {}
        for act in self._all_actions:
            new_seq = seq_predi + [act]
            seq_to_predict = seq + [tokenizer.encode(act)]
            seq_to_predict = torch.tensor(seq_to_predict, dtype=torch.float).to(device)
            x = self._model(seq_to_predict)
            # Transforme x into list proba
            probs = torch.nn.functional.softmax(x, dim=0).tolist()
            # for each outcomes we want proba with act
            for i, out in enumerate(self._all_outcomes):
                tmp_new_seq = new_seq + [out]
                tmp_proba = probs[i] * proba
                if tmp_proba < seuil:
                    continue
                tempo = float(np.round(self._valence[inter(act, out)] * tmp_proba, decimals=4))
                # input(f'seq {seq_predi} act {act} out {out} proba {tmp_proba} valance {valance[(act, out)]} tempo {tempo}')
                exceptive_valance.update(
                    self.recursif_expective_valance(seq=seq[2:] + [tokenizer.encode(act), tokenizer.encode(out)],
                                                max_depth=max_depth, seuil=seuil, 
                                                proba=tmp_proba, seq_predi=tmp_new_seq.copy())
                )
                exceptive_valance[str(tmp_new_seq)] = tempo
        return exceptive_valance
        

    def decide(self):
        gap = (self._model.input_size - 1) // 2
        x = []
        for i in range(len(self._history_act) - gap, len(self._history_act)):
            x.append(self._history_act[i])
            x.append(self._history_fb[i])
        seq = self._tokenizer.encode(x)
        res = self.recursif_expective_valance(seq=seq, max_depth=3, seuil=0.2)
        top_5 = sorted(res.items(), key=lambda x: x[1], reverse=True)[:5]
        print(f"Top 5 of sequences with the best expected valance for {x}")
        for top in top_5:
            print(f"Sequence: {top[0]} Expected valance: {top[1]}")
        
        print(f"Action choisie : {eval(top_5[0][0])[0]}")
        return eval(top_5[0][0])[0]


    # Modifier
    def action(self, outcome, decide=True, validate_loader=None):
        """ 
        Fonction qui choisit l'action a faire en fonction de la dernière \
        intéraction avec l'environnement. \n
        C'est ici que nous allons implémenter un mécanisme de ML \
        pour choisir la prochaine action.

        :param: **outcome** feedback de la dernière intéraction avec l'environnement

        :return: **action** action à effectuer
        """
        if self._action is not None:
            self._history_fb.append(outcome)
            print(f"Action: {self._action}, Prediction: {self._predicted_outcome}, Outcome: {outcome}, " 
                  f"\033[0;31m Satisfaction: {self._predicted_outcome == outcome} \033[0m")
            if self._predicted_outcome != outcome:
                if len(self._history_act) + len(self._history_fb) > self._model.input_size:
                    self.fit(self._history_act, self._history_fb, validate_loader=validate_loader)
                else:
                    self._action = self._all_actions[0]

            # Maintenant nous choisissons la prochaine action en fonction de la valance
            if decide:
                self._action = self.decide()
            else :
                self._action = str(np.random.choice(self._all_actions))
            if len(self._history_act) + len(self._history_fb) + 1 > self._model.input_size:
                self._predicted_outcome = self.predict(self._action)
            self._history_act.append(self._action)
        else:
            self._action = self._all_actions[0]
            self._history_act.append(self._action)            
            print(f"Action de base : {self._action} Prediction: {self._predicted_outcome}")
        
        return self._action, self._predicted_outcome

In [10]:
out = Output()
EnvDisplay = WorldDisplay()

In [11]:
env_test2 = small_loop(x=1, y=1, theta=0)

model_ML = DeepNetwork(hidden_size=[16], input_size=3, output_size=2)
optimizer = torch.optim.Adam(model_ML.parameters(), lr=0.01, weight_decay=0.001)
loss_func = nn.CrossEntropyLoss()
tokenizer = SimpleTokenizerV1(create_dico_numerate_word(env_test2.get_outcomes() + env_test2.get_actions()))

valence = {
    inter('forward', 'empty') : 1,
    inter('forward', 'wall') : -10,
    inter('turn_left', 'empty') : -3,
    inter('turn_left', 'wall') : -3,
    inter('turn_right', 'empty') : -3,
    inter('turn_right', 'wall') : -3,
    inter('feel_front', 'wall') : -5,
    inter('feel_front', 'empty') : -5,
}

agent_test2 = AgentPEBis(
    model=model_ML,
    all_outcomes= env_test2.get_outcomes(),
    all_actions= env_test2.get_actions(),
    valence=valence,
    tokenizer=tokenizer,
    optimizer=optimizer,
    loss_func=loss_func)

history_good = []
pourcent_by_10 = []
outcome = None
for i in range(10):
    print(f"=======================\033[0;32m iteration {i} \033[0m=======================")
    action, predi = agent_test2.action(outcome, False)
    outcome = env_test2.outcome(action)
    history_good.append(outcome == predi)
    pourcent_by_10.append(sum(history_good[-10:]) * 10 if len(history_good) >= 10 else 0)
    print(f'action {action} predi {predi} outcome {outcome}')
    print(f"Action choisie : {action} \033[0;34m{pourcent_by_10[-1]} \033[0m")
    print("\n")

# On l'entraine 200 fois



for i in range(100):
    print(f"=======================\033[0;32m iteration {i} \033[0m=======================")
    action, predi = agent_test2.action(outcome, decide=True)
    outcome = env_test2.outcome(action)
    history_good.append(outcome == predi)
    pourcent_by_10.append(sum(history_good[-10:]) * 10 if len(history_good) >= 10 else 0)
    print(f'action {action} predi {predi} outcome {outcome}')
    print(f"Action choisie : {action} \033[0;34m{pourcent_by_10[-1]} \033[0m")
    print("\n")
    # sleep(0.5)
    # EnvDisplay.show(env_test2.get_world(), env_test2.get_robot(), out)
    # display(out)
    # time.sleep(0.5)
    

liste hidden init [16]
======================= iteration 0 =======================
Action de base : forward Prediction: None
action forward predi None outcome wall
Action choisie : forward 0 


======================= iteration 1 =======================
Action: forward, Prediction: None, Outcome: wall,  Satisfaction: False 
action forward predi None outcome wall
Action choisie : forward 0 


======================= iteration 2 =======================
Action: forward, Prediction: None, Outcome: wall,  Satisfaction: False 
Epoch 1/20, Loss: 4.5089
Epoch 6/20, Loss: 2.3296
Epoch 11/20, Loss: 1.0271
Epoch 16/20, Loss: 0.4452
action turn_left predi wall outcome empty
Action choisie : turn_left 0 


======================= iteration 3 =======================
Action: turn_left, Prediction: wall, Outcome: empty,  Satisfaction: False 
Epoch 1/20, Loss: 1.1745
Epoch 6/20, Loss: 1.3393
Epoch 11/20, Loss: 1.1074
Epoch 16/20, Loss: 0.7854
action forward predi wall outcome wall
Action choisie : forw

# Résultat
Nous voyons un problème apparaître, le modèl prédit à 49% 'forward', 'empty' parce qu'il ne s'est pas assez entrainner. Donc en espected valence le choix forward aurais bcp d'importance alors qu'il y a peu de possibilité.

Il y a un autre problème, le modèle ne fais pas de différence entre `turn left` et `turn right`, nous avons vue que le modèle avec le mécanisme de décision pouvais choisire de faire l'un puis l'autre avant de nouveau faire `forward`. Ce qui n'a pas de sens.

## Solution
Nous pouvons données plus d'epoch lors du fit. Mais le problème risque toujours de se poser si l'environement change ou si un pattern d'interaction est sous exploité.

In [12]:
class Agentbored:
    def __init__(self, model, all_outcomes, all_actions, tokenizer, valence:dict,
                optimizer=None, loss_func=None):
        """ 
        Création de l'agent.
        
        - self._action : action précédente
        - self._predicted_outcome : prédiction de l'outcome précédent
        """
        self._action = None
        self._predicted_outcome = None
        self._model = model
        self._otimizer = optimizer
        self._loss_func = loss_func
        self._tokenizer:SimpleTokenizerV1 = tokenizer
        self._all_outcomes = all_outcomes
        self._all_actions = all_actions
        self._history_act = []
        self._history_fb = []
        self._valence = valence
        self._bored = 0

    def fit(self, actions:list, outcomes:list, nb_epoch:int= 5, validate_loader=None):
        """
        Fonction d'entrainement de l'agent 
        Avec data set custom, le model prends en inputs plusieurs données
        """
        context_lenght = self._model.input_size
        if len(actions) + len(outcomes) < context_lenght:
            raise Exception("Not enough data to train model")

        actions = [self._tokenizer.encode(act) for act in actions]
        outcomes = self._tokenizer.encode(outcomes)
        
        if isinstance(self._model, torch.nn.Module):
            self._model.train()
            data_loarder = CustomDataSet(actions=actions, outcomes=outcomes,
                         context_lenght=context_lenght, dim_out=len(self._all_outcomes))

            data_loader = torch.utils.data.DataLoader(
                data_loarder,batch_size=32, shuffle=True)

            train_with_batch(model=self._model, 
                    train_loader=data_loader,
                    optimizer=self._otimizer,
                    loss_func=self._loss_func,
                    nb_epochs=nb_epoch,
                    validate_loader=validate_loader,
                    print_=False)
        else: # Si le model n'est pas un model pytorch
            raise Exception('Not implemented')
            self._model.fit(action, outcome)
            pass

    def get_prediction(self, action):
        gap = (self._model.input_size - 1) // 2
        x = []
        for i in range(len(self._history_act) - gap, len(self._history_act)):
            x.append(self._history_act[i])
            x.append(self._history_fb[i])
        x.append(action)
        action = self._tokenizer.encode(x)
        
        if isinstance(self._model, torch.nn.Module):
            self._model.eval() 
            action = torch.tensor(action, dtype=torch.float).to(device)
            x = self._model(action)
            x = torch.nn.functional.softmax(x, dim=0)
        else:
            raise Exception('Not implemented')
            x=self._model.predict(action)
        
        return x

    def predict(self, action):
        """
        Funciton de prédiction
        """

        if len(self._history_act) + len(self._history_fb) + 1 < self._model.input_size:
            raise Exception("Not enough data to train model")
        
        gap = (self._model.input_size - 1) // 2
        x = []
        for i in range(len(self._history_act) - gap, len(self._history_act)):
            x.append(self._history_act[i])
            x.append(self._history_fb[i])
        x.append(action)
        action = self._tokenizer.encode(x)
        if isinstance(self._model, torch.nn.Module):
            self._model.eval() 
            action = torch.tensor(action, dtype=torch.float).to(device)
            x = self._model(action)
            x = torch.argmax(x, dim=0).item()

        else:
            raise Exception('Not implemented')
            x=self._model.predict(action)
        
        return self._tokenizer.decode(x)
    
    def recursif_expective_valance(self, seq:list, max_depth:int, seuil:float=0.2, proba:float = 1,
                                    seq_predi:list = []):
        """
        """
        if max_depth == 0:
            return {}
        max_depth -= 1

        if proba < seuil:
            return {}
        
        self._model.eval()
        exceptive_valance = {}
        for act in self._all_actions:
            new_seq = seq_predi + [act]
            seq_to_predict = seq + [tokenizer.encode(act)]
            seq_to_predict = torch.tensor(seq_to_predict, dtype=torch.float).to(device)
            x = self._model(seq_to_predict)
            # Transforme x into list proba
            probs = torch.nn.functional.softmax(x, dim=0).tolist()
            # for each outcomes we want proba with act
            for i, out in enumerate(self._all_outcomes):
                tmp_new_seq = new_seq + [out]
                tmp_proba = probs[i] * proba
                if tmp_proba < seuil:
                    continue
                tempo = float(np.round(self._valence[inter(act, out)] * tmp_proba, decimals=4))
                # input(f'seq {seq_predi} act {act} out {out} proba {tmp_proba} valance {valance[(act, out)]} tempo {tempo}')
                exceptive_valance.update(
                    self.recursif_expective_valance(seq=seq[2:] + [tokenizer.encode(act), tokenizer.encode(out)],
                                                max_depth=max_depth, seuil=seuil, 
                                                proba=tmp_proba, seq_predi=tmp_new_seq.copy())
                )
                exceptive_valance[str(tmp_new_seq)] = tempo
        return exceptive_valance
        
    def decide(self):
        gap = (self._model.input_size - 1) // 2
        x = []
        for i in range(len(self._history_act) - gap, len(self._history_act)):
            x.append(self._history_act[i])
            x.append(self._history_fb[i])
        seq = self._tokenizer.encode(x)
        res = self.recursif_expective_valance(seq=seq, max_depth=3, seuil=0.4)
        top_5 = sorted(res.items(), key=lambda x: x[1], reverse=True)[:5]
        # print(f"Top 5 of sequences with the best expected valance for {x}")
        # for top in top_5:
        #     print(f"Sequence: {top[0]} Expected valance: {top[1]}")
        
        # print(f"Action choisie : {eval(top_5[0][0])[0]}")
        return eval(top_5[0][0])[0]

    # Modifier
    def action(self, outcome, decide=True, validate_loader=None):
        """ 
        Fonction qui choisit l'action a faire en fonction de la dernière \
        intéraction avec l'environnement. \n
        C'est ici que nous allons implémenter un mécanisme de ML \
        pour choisir la prochaine action.

        :param: **outcome** feedback de la dernière intéraction avec l'environnement

        :return: **action** action à effectuer
        """
        if self._action is not None:
            self._history_fb.append(outcome)
            print(f"Action: {self._action}, Prediction: {self._predicted_outcome}, Outcome: {outcome}, " 
                  f"\033[0;31m Satisfaction: {self._predicted_outcome == outcome} \033[0m")
            if len(self._history_act) + len(self._history_fb) > self._model.input_size:
                # print("j'ai assez de données pour entrainer")
                if self._predicted_outcome != outcome:
                    self.fit(self._history_act, self._history_fb, validate_loader=validate_loader)
                    self._bored = 0
                else:
                    self._action = self._all_actions[0]
                if self._bored >= 10: # Ajout de bored qui permet de relancer l'entrainement
                    self._bored = 0
                    self.fit(self._history_act, self._history_fb, validate_loader=validate_loader)
                    decide = False
            if decide:
                self._action = self.decide()
            else :
                self._action = str(np.random.choice(self._all_actions))
            if len(self._history_act) + len(self._history_fb) + 1 > self._model.input_size:
                self._predicted_outcome = self.predict(self._action)
            self._history_act.append(self._action)
            self._bored += 1
        else:
            self._action = self._all_actions[0]
            self._history_act.append(self._action)            
            # print(f"Action de base : {self._action} Prediction: {self._predicted_outcome}")
        
        return self._action, self._predicted_outcome

In [27]:
env_test2 = small_loop(x=1, y=1, theta=0)

model_ML = DeepNetwork(hidden_size=[16], input_size=3, output_size=2)
optimizer = torch.optim.Adam(model_ML.parameters(), lr=0.001, weight_decay=0.0001)
loss_func = nn.CrossEntropyLoss()
tokenizer = SimpleTokenizerV1(create_dico_numerate_word(env_test2.get_outcomes() + env_test2.get_actions()))

valence = {
    inter('forward', 'empty') : 5,
    inter('forward', 'wall') : -60,
    inter('turn_left', 'empty') : -7,
    inter('turn_left', 'wall') : -7,
    inter('turn_right', 'empty') : -7,
    inter('turn_right', 'wall') : -7,
    inter('feel_front', 'wall') : -5,
    inter('feel_front', 'empty') : -5,
}

agent_test2 = Agentbored(
    model=model_ML,
    all_outcomes= env_test2.get_outcomes(),
    all_actions= env_test2.get_actions(),
    valence=valence,
    tokenizer=tokenizer,
    optimizer=optimizer,
    loss_func=loss_func)

history_good = []
pourcent_by_10 = []
outcome = None
for i in range(100):
    print(f"=======================\033[0;32m iteration {i} \033[0m=======================")
    action, predi = agent_test2.action(outcome, False)
    outcome = env_test2.outcome(action)
    history_good.append(outcome == predi)
    pourcent_by_10.append(sum(history_good[-10:]) * 10 if len(history_good) >= 10 else 0)
    print(f'action {action} predi {predi} outcome {outcome}')
    print(f"Action choisie : {action} \033[0;34m{pourcent_by_10[-1]} \033[0m")
    print("\n")

for i in range(100):
    print(f"=======================\033[0;32m iteration {i} \033[0m=======================")
    action, predi = agent_test2.action(outcome, decide=True)
    outcome = env_test2.outcome(action)
    history_good.append(outcome == predi)
    pourcent_by_10.append(sum(history_good[-10:]) * 10 if len(history_good) >= 10 else 0)
    # print(f'action {action} predi {predi} outcome {outcome}')
    # print(f"Action choisie : {action} \033[0;34m{pourcent_by_10[-1]} \033[0m")
    # print("\n")
    # sleep(0.5)

    

liste hidden init [16]
======================= iteration 0 =======================
action forward predi None outcome wall
Action choisie : forward 0 


======================= iteration 1 =======================
Action: forward, Prediction: None, Outcome: wall,  Satisfaction: False 
action turn_right predi None outcome empty
Action choisie : turn_right 0 


======================= iteration 2 =======================
Action: turn_right, Prediction: None, Outcome: empty,  Satisfaction: False 
action turn_right predi wall outcome empty
Action choisie : turn_right 0 


======================= iteration 3 =======================
Action: turn_right, Prediction: wall, Outcome: empty,  Satisfaction: False 
action turn_right predi wall outcome empty
Action choisie : turn_right 0 


======================= iteration 4 =======================
Action: turn_right, Prediction: wall, Outcome: empty,  Satisfaction: False 
action turn_right predi wall outcome empty
Action choisie : turn_right 0 


====

In [14]:
action, predi = agent_test2.action(outcome, decide=True)
outcome = env_test2.outcome(action)
history_good.append(outcome == predi)
pourcent_by_10.append(sum(history_good[-10:]) * 10 if len(history_good) >= 10 else 0)
# print(f'action {action} predi {predi} outcome {outcome}')
# print(f"Action choisie : {action} \033[0;34m{pourcent_by_10[-1]} \033[0m")
# print("\n")
# sleep(0.5)
EnvDisplay.show(env_test2.get_world(), env_test2.get_robot(), out)
display(out)

Action: forward, Prediction: empty, Outcome: empty,  Satisfaction: True 


Output()

In [15]:
raise Exception("Fin ")

Exception: Fin 

In [28]:
def show_proba_from_seq(_seg):
    seq = tokenizer.encode(_seg)
    seq = torch.tensor(seq, dtype=torch.float).to(device)
    predi =  agent_test2._model(seq)
    prob = torch.nn.functional.softmax(predi, dim=0)
    deocde = tokenizer.decode(torch.argmax(predi, dim=0).item())
    print(f"Prédiction de la séquence {_seg} :  probabilité {prob.tolist()}, decode {deocde}")

print("porba si on avance")
show_proba_from_seq(['forward', 'empty', 'forward'])
show_proba_from_seq(['forward', 'wall', 'forward'])
show_proba_from_seq(['turn_left', 'empty', 'forward'])
show_proba_from_seq(['turn_right', 'empty', 'forward'])
show_proba_from_seq(['feel_front', 'empty', 'forward'])
show_proba_from_seq(['feel_front', 'wall', 'forward'])

print("porba si on turn left")
show_proba_from_seq(['forward', 'empty', 'turn_left'])
show_proba_from_seq(['forward', 'wall', 'turn_left'])
show_proba_from_seq(['turn_left', 'empty', 'turn_left'])
show_proba_from_seq(['turn_right', 'empty', 'turn_left'])
show_proba_from_seq(['feel_front', 'empty', 'turn_left'])
show_proba_from_seq(['feel_front', 'wall', 'turn_left'])

print("porba si on turn right")
show_proba_from_seq(['forward', 'empty', 'turn_right'])
show_proba_from_seq(['forward', 'wall', 'turn_right'])
show_proba_from_seq(['turn_left', 'empty', 'turn_right'])
show_proba_from_seq(['turn_right', 'empty', 'turn_right'])
show_proba_from_seq(['feel_front', 'empty', 'turn_right'])
show_proba_from_seq(['feel_front', 'wall', 'turn_right'])

print("porba si on feel front")
show_proba_from_seq(['forward', 'empty', 'feel_front'])
show_proba_from_seq(['forward', 'wall', 'feel_front'])
show_proba_from_seq(['turn_left', 'empty', 'feel_front'])
show_proba_from_seq(['turn_right', 'empty', 'feel_front'])
show_proba_from_seq(['feel_front', 'empty', 'feel_front'])
show_proba_from_seq(['feel_front', 'wall', 'feel_front'])

# Save in file txt agent._histoty_act and agent._history_fb

# with open('history_agent.txt', 'w') as f:
#     list_to_save = zip(agent_test2._history_act, agent_test2._history_fb)
#     # for x, y in list_to_save:
#     #     # f.write(str(f"pattern {x}, {y}; ")) 
#     #     f.write("\n")
#     f.write(str(agent_test2._history_act))
#     f.write("\n")
#     f.write(str(agent_test2._history_fb))
#     f.write("\n")

def count_pattern(lst, pattern):
    return sum(1 for i in range(len(lst) - len(pattern) + 1) if lst[i:i+len(pattern)] == pattern)

list_act_out = []
for act, out in zip(agent_test2._history_act, agent_test2._history_fb):
    list_act_out.append(act)
    list_act_out.append(out)

print("count si on avance")
print(f"pattern 'forward', 'empty', 'forward', 'empty' présent : {count_pattern(list_act_out, ['forward', 'empty', 'forward', 'empty'])}")
print(f"pattern 'forward', 'empty', 'forward', 'wall' présent : {count_pattern(list_act_out, ['forward', 'empty', 'forward', 'wall'])}")
print(f"pattern 'forward', 'wall', 'forward', 'empty' présent : {count_pattern(list_act_out, ['forward', 'wall', 'forward', 'empty'])}")
print(f"pattern 'forward', 'wall', 'forward', 'wall' présent : {count_pattern(list_act_out, ['forward', 'wall', 'forward', 'wall'])}")
print(f"pattern 'turn_left', 'empty', 'forward', 'empty' présent : {count_pattern(list_act_out, ['turn_left', 'empty', 'forward', 'empty'])}")
print(f"pattern 'turn_left', 'empty', 'forward', 'wall' présent : {count_pattern(list_act_out, ['turn_left', 'empty', 'forward', 'wall'])}")
print(f"pattern 'turn_right', 'empty', 'forward', 'empty' présent : {count_pattern(list_act_out, ['turn_right', 'empty', 'forward', 'empty'])}")
print(f"pattern 'turn_right', 'empty', 'forward', 'wall' présent : {count_pattern(list_act_out, ['turn_right', 'empty', 'forward', 'wall'])}")
print(f"pattern 'feel_front', 'empty', 'forward', 'empty' présent : {count_pattern(list_act_out, ['feel_front', 'empty', 'forward', 'empty'])}")
print(f"pattern 'feel_front', 'empty', 'forward', 'wall' présent : {count_pattern(list_act_out, ['feel_front', 'empty', 'forward', 'wall'])}")
print(f"pattern 'feel_front', 'wall', 'forward', 'empty' présent : {count_pattern(list_act_out, ['feel_front', 'wall', 'forward', 'empty'])}")
print(f"pattern 'feel_front', 'wall', 'forward', 'wall' présent : {count_pattern(list_act_out, ['feel_front', 'wall', 'forward', 'wall'])}")

print("count si on feel_front")
print(f"pattern 'forward', 'empty', 'feel_front', 'empty' présent : {count_pattern(list_act_out, ['forward', 'empty', 'feel_front', 'empty'])}")
print(f"pattern 'forward', 'empty', 'feel_front', 'wall' présent : {count_pattern(list_act_out, ['forward', 'empty', 'feel_front', 'wall'])}")
print(f"pattern 'forward', 'wall', 'feel_front', 'empty' présent : {count_pattern(list_act_out, ['forward', 'wall', 'feel_front', 'empty'])}")
print(f"pattern 'forward', 'wall', 'feel_front', 'wall' présent : {count_pattern(list_act_out, ['forward', 'wall', 'feel_front', 'wall'])}")
print(f"pattern 'turn_left', 'empty', 'feel_front', 'empty' présent : {count_pattern(list_act_out, ['turn_left', 'empty', 'feel_front', 'empty'])}")
print(f"pattern 'turn_left', 'empty', 'feel_front', 'wall' présent : {count_pattern(list_act_out, ['turn_left', 'empty', 'feel_front', 'wall'])}")
print(f"pattern 'turn_right', 'empty', 'feel_front', 'empty' présent : {count_pattern(list_act_out, ['turn_right', 'empty', 'feel_front', 'empty'])}")
print(f"pattern 'turn_right', 'empty', 'feel_front', 'wall' présent : {count_pattern(list_act_out, ['turn_right', 'empty', 'feel_front', 'wall'])}")
print(f"pattern 'feel_front', 'empty', 'feel_front', 'empty' présent : {count_pattern(list_act_out, ['feel_front', 'empty', 'feel_front', 'empty'])}")
print(f"pattern 'feel_front', 'empty', 'feel_front', 'wall' présent : {count_pattern(list_act_out, ['feel_front', 'empty', 'feel_front', 'wall'])}")
print(f"pattern 'feel_front', 'wall', 'feel_front', 'empty' présent : {count_pattern(list_act_out, ['feel_front', 'wall', 'feel_front', 'empty'])}")
print(f"pattern 'feel_front', 'wall', 'feel_front', 'wall' présent : {count_pattern(list_act_out, ['feel_front', 'wall', 'feel_front', 'wall'])}")

print("count si on turn_left turn_right")
print(f"pattern 'turn_left', 'empty', 'turn_left', 'empty' présent : {count_pattern(list_act_out, ['turn_left', 'empty', 'turn_left', 'empty'])}")
print(f"pattern 'turn_right', 'empty', 'turn_left', 'empty' présent : {count_pattern(list_act_out, ['turn_right', 'empty', 'turn_left', 'empty'])}")
print(f"pattern 'turn_right', 'empty', 'turn_right', 'empty' présent : {count_pattern(list_act_out, ['turn_right', 'empty', 'turn_right', 'empty'])}")
print(f"pattern 'turn_left', 'empty', 'turn_right', 'empty' présent : {count_pattern(list_act_out, ['turn_left', 'empty', 'turn_right', 'empty'])}")



porba si on avance
Prédiction de la séquence ['forward', 'empty', 'forward'] :  probabilité [0.18715326488018036, 0.8128467798233032], decode empty
Prédiction de la séquence ['forward', 'wall', 'forward'] :  probabilité [0.7130916118621826, 0.2869083881378174], decode wall
Prédiction de la séquence ['turn_left', 'empty', 'forward'] :  probabilité [0.15437881648540497, 0.8456212282180786], decode empty
Prédiction de la séquence ['turn_right', 'empty', 'forward'] :  probabilité [0.19474142789840698, 0.805258572101593], decode empty
Prédiction de la séquence ['feel_front', 'empty', 'forward'] :  probabilité [0.24262908101081848, 0.7573708891868591], decode empty
Prédiction de la séquence ['feel_front', 'wall', 'forward'] :  probabilité [0.7548562288284302, 0.24514377117156982], decode wall
porba si on turn left
Prédiction de la séquence ['forward', 'empty', 'turn_left'] :  probabilité [0.2406865805387497, 0.7593134641647339], decode empty
Prédiction de la séquence ['forward', 'wall', 'tur

Randome + decision
count si on avance
pattern 'forward', 'empty', 'forward', 'empty' présent : 4
pattern 'forward', 'empty', 'forward', 'wall' présent : 5
pattern 'forward', 'wall', 'forward', 'empty' présent : 0
pattern 'forward', 'wall', 'forward', 'wall' présent : 48
pattern 'turn_left', 'empty', 'forward', 'empty' présent : 5
pattern 'turn_left', 'empty', 'forward', 'wall' présent : 1
pattern 'turn_right', 'empty', 'forward', 'empty' présent : 7
pattern 'turn_right', 'empty', 'forward', 'wall' présent : 5
pattern 'feel_front', 'empty', 'forward', 'empty' présent : 4
pattern 'feel_front', 'empty', 'forward', 'wall' présent : 0
pattern 'feel_front', 'wall', 'forward', 'empty' présent : 0
pattern 'feel_front', 'wall', 'forward', 'wall' présent : 15
count si on feel_front
pattern 'forward', 'empty', 'feel_front', 'empty' présent : 1
pattern 'forward', 'empty', 'feel_front', 'wall' présent : 2
pattern 'forward', 'wall', 'feel_front', 'empty' présent : 0
pattern 'forward', 'wall', 'feel_front', 'wall' présent : 17
pattern 'turn_left', 'empty', 'feel_front', 'empty' présent : 5
pattern 'turn_left', 'empty', 'feel_front', 'wall' présent : 6
pattern 'turn_right', 'empty', 'feel_front', 'empty' présent : 3
pattern 'turn_right', 'empty', 'feel_front', 'wall' présent : 3
pattern 'feel_front', 'empty', 'feel_front', 'empty' présent : 0
pattern 'feel_front', 'empty', 'feel_front', 'wall' présent : 0
pattern 'feel_front', 'wall', 'feel_front', 'empty' présent : 0
pattern 'feel_front', 'wall', 'feel_front', 'wall' présent : 1
count si on turn_left turn_right
pattern 'turn_left', 'empty', 'turn_left', 'empty' présent : 3
pattern 'turn_right', 'empty', 'turn_left', 'empty' présent : 10
pattern 'turn_right', 'empty', 'turn_right', 'empty' présent : 11
pattern 'turn_left', 'empty', 'turn_right', 'empty' présent : 7

Juste randome

count si on avance
- pattern 'forward', 'empty', 'forward', 'empty' présent : 2
- pattern 'forward', 'empty', 'forward', 'wall' présent : 0
- pattern 'forward', 'wall', 'forward', 'empty' présent : 0
- pattern 'forward', 'wall', 'forward', 'wall' présent : 3
- pattern 'turn_left', 'empty', 'forward', 'empty' présent : 3
- pattern 'turn_left', 'empty', 'forward', 'wall' présent : 2
- pattern 'turn_right', 'empty', 'forward', 'empty' présent : 3
- pattern 'turn_right', 'empty', 'forward', 'wall' présent : 6
- pattern 'feel_front', 'empty', 'forward', 'empty' présent : 2
- pattern 'feel_front', 'empty', 'forward', 'wall' présent : 0
- pattern 'feel_front', 'wall', 'forward', 'empty' présent : 0
- pattern 'feel_front', 'wall', 'forward', 'wall' présent : 4

count si on feel_front
- pattern 'forward', 'empty', 'feel_front', 'empty' présent : 1
- pattern 'forward', 'empty', 'feel_front', 'wall' présent : 2
- pattern 'forward', 'wall', 'feel_front', 'empty' présent : 0
- pattern 'forward', 'wall', 'feel_front', 'wall' présent : 4
- pattern 'turn_left', 'empty', 'feel_front', 'empty' présent : 0
- pattern 'turn_left', 'empty', 'feel_front', 'wall' présent : 2
- pattern 'turn_right', 'empty', 'feel_front', 'empty' présent : 3
- pattern 'turn_right', 'empty', 'feel_front', 'wall' présent : 4
- pattern 'feel_front', 'empty', 'feel_front', 'empty' présent : 0
- pattern 'feel_front', 'empty', 'feel_front', 'wall' présent : 0
- pattern 'feel_front', 'wall', 'feel_front', 'empty' présent : 0
- pattern 'feel_front', 'wall', 'feel_front', 'wall' présent : 8

count si on turn_left turn_right
- pattern 'turn_left', 'empty', 'turn_left', 'empty' présent : 8
- pattern 'turn_right', 'empty', 'turn_left', 'empty' présent : 4
- pattern 'turn_right', 'empty', 'turn_right', 'empty' présent : 5
- pattern 'turn_left', 'empty', 'turn_right', 'empty' présent : 8

# Probabilité obtenut après 500 itération :
valence = {
    inter('forward', 'empty') : 1,
    inter('forward', 'wall') : -30,
    inter('turn_left', 'empty') : -5,
    inter('turn_left', 'wall') : -5,
    inter('turn_right', 'empty') : -5,
    inter('turn_right', 'wall') : -5,
    inter('feel_front', 'wall') : -5,
    inter('feel_front', 'empty') : -5,
}
## porba si on avance
- Prédiction de la séquence ['forward', 'empty', 'forward'] :  probabilité [0.49284496903419495, 0.5071550607681274], decode empty  
    Sur cette séquence, il est normal que le modèl ne soit pas sur. Mais a cause de du mécansime de décision on peut s'attendre a ce que cette probabilité soit très variable
- Prédiction de la séquence ['forward', 'wall', 'forward'] :  probabilité [0.9998795986175537, 0.00012042034359183162], decode wall  
    Ici la prédiciton est logique et sur
- Prédiction de la séquence ['turn_left', 'empty', 'forward'] :  probabilité [0.2555951178073883, 0.7444049119949341], decode empty  
    Pour cette sequence on voit l'effet négatif du mécanisme de décision. Vue que le robot ne fais plus d'action aléatoire certain pattern deviennent sous représenté et d'autre au contraire sont trop présent (je reviens sur ce phénomen après) 
- Prédiction de la séquence ['turn_right', 'empty', 'forward'] :  probabilité [0.10819040238857269, 0.8918095827102661], decode empty  
    Cette séquence prouve ce que je raconte au dessus, l'action tourné a droite a plus souvent aboutit sur empty.
- Prédiction de la séquence ['feel_front', 'empty', 'forward'] :  probabilité [0.04110181704163551, 0.9588981866836548], decode empty  
    Séquence parfaite et pertinent pour l'agent.
- Prédiction de la séquence ['feel_front', 'wall', 'forward'] :  probabilité [0.9806479811668396, 0.019352057948708534], decode wall

## porba si on turn left
- Prédiction de la séquence ['forward', 'empty', 'turn_left'] :  probabilité [0.2387831062078476, 0.7612168788909912], decode empty  
    étonnant, le modèl n'a pas encore compris que `turn left` (ou `turn right`) renvois tout le temps empty 
- Prédiction de la séquence ['forward', 'wall', 'turn_left'] :  probabilité [0.9999569654464722, 4.300793443690054e-05], decode wall  
    Abérent
- Prédiction de la séquence ['turn_left', 'empty', 'turn_left'] :  probabilité [0.023873118683695793, 0.9761269092559814], decode empty  
- Prédiction de la séquence ['turn_right', 'empty', 'turn_left'] :  probabilité [0.000307118782075122, 0.9996929168701172], decode empty  
- Prédiction de la séquence ['feel_front', 'empty', 'turn_left'] :  probabilité [0.00010853375715669245, 0.9998914003372192], decode empty  
- Prédiction de la séquence ['feel_front', 'wall', 'turn_left'] :  probabilité [0.11550579220056534, 0.8844942450523376], decode empty  


## porba si on turn right
- Prédiction de la séquence ['forward', 'empty', 'turn_right'] :  probabilité [0.42596185207366943, 0.5740381479263306], decode empty  
- Prédiction de la séquence ['forward', 'wall', 'turn_right'] :  probabilité [0.9999817609786987, 1.8181275663664564e-05], decode wall  
    Abérent
- Prédiction de la séquence ['turn_left', 'empty', 'turn_right'] :  probabilité [0.1661546677350998, 0.8338453769683838], decode empty  
- Prédiction de la séquence ['turn_right', 'empty', 'turn_right'] :  probabilité [0.0050218072719872, 0.994978129863739], decode empty  
- Prédiction de la séquence ['feel_front', 'empty', 'turn_right'] :  probabilité [3.543415732565336e-05, 0.9999645948410034], decode empty  
- Prédiction de la séquence ['feel_front', 'wall', 'turn_right'] :  probabilité [0.7933242321014404, 0.20667579770088196], decode wall  

## porba si on feel front
- Prédiction de la séquence ['forward', 'empty', 'feel_front'] :  probabilité [0.6370701789855957, 0.3629298508167267], decode wall  
    Pourquoi pas, mais toujours sensible aux interactions faites avant.
- Prédiction de la séquence ['forward', 'wall', 'feel_front'] :  probabilité [0.9999923706054688, 7.685893251618836e-06], decode wall  
    C'est ce qu'on veut
- Prédiction de la séquence ['turn_left', 'empty', 'feel_front'] :  probabilité [0.3203613758087158, 0.679638683795929], decode empty  
    Pourquoi pas, mais toujours sensible aux interactions faites avant.
- Prédiction de la séquence ['turn_right', 'empty', 'feel_front'] :  probabilité [0.11235635727643967, 0.8876436352729797], decode empty  
    Pourquoi pas, mais toujours sensible aux interactions faites avant.
- Prédiction de la séquence ['feel_front', 'empty', 'feel_front'] :  probabilité [0.0010404880158603191, 0.9989595413208008], decode empty  
    C'est ce qu'on veut
- Prédiction de la séquence ['feel_front', 'wall', 'feel_front'] :  probabilité [0.9912147521972656, 0.008785244077444077], decode wall  
    C'est ce qu'on veut

In [ ]:
env_test2 = small_loop(x=1, y=1, theta=0)

model_ML = DeepNetwork(hidden_size=[16], input_size=3, output_size=2)
optimizer = torch.optim.Adam(model_ML.parameters(), lr=0.001, weight_decay=0.0001)
loss_func = nn.CrossEntropyLoss()
tokenizer = SimpleTokenizerV1(create_dico_numerate_word(env_test2.get_outcomes() + env_test2.get_actions()))

valence = {
    inter('forward', 'empty') : 1,
    inter('forward', 'wall') : -30,
    inter('turn_left', 'empty') : -10,
    inter('turn_left', 'wall') : -10,
    inter('turn_right', 'empty') : -10,
    inter('turn_right', 'wall') : -10,
    inter('feel_front', 'wall') : -5,
    inter('feel_front', 'empty') : -5,
}

agent_test2 = Agentbored(
    model=model_ML,
    all_outcomes= env_test2.get_outcomes(),
    all_actions= env_test2.get_actions(),
    valence=valence,
    tokenizer=tokenizer,
    optimizer=optimizer,
    loss_func=loss_func)

history_good = []
pourcent_by_10 = []
outcome = None
for i in range(10):
    print(f"=======================\033[0;32m iteration {i} \033[0m=======================")
    action, predi = agent_test2.action(outcome, False)
    outcome = env_test2.outcome(action)
    history_good.append(outcome == predi)
    pourcent_by_10.append(sum(history_good[-10:]) * 10 if len(history_good) >= 10 else 0)
    print(f'action {action} predi {predi} outcome {outcome}')
    print(f"Action choisie : {action} \033[0;34m{pourcent_by_10[-1]} \033[0m")
    print("\n")

for i in range(500):
    print(f"=======================\033[0;32m iteration {i} \033[0m=======================")
    action, predi = agent_test2.action(outcome, decide=True)
    outcome = env_test2.outcome(action)
    history_good.append(outcome == predi)
    pourcent_by_10.append(sum(history_good[-10:]) * 10 if len(history_good) >= 10 else 0)
    print(f'action {action} predi {predi} outcome {outcome}')
    print(f"Action choisie : {action} \033[0;34m{pourcent_by_10[-1]} \033[0m")
    print("\n")
    # sleep(0.5)
    # EnvDisplay.show(env_test2.get_world(), env_test2.get_robot(), out)
    # display(out)
    # time.sleep(0.5)
    

In [ ]:
def show_proba_from_seq(_seg):
    seq = tokenizer.encode(_seg)
    seq = torch.tensor(seq, dtype=torch.float).to(device)
    predi =  agent_test2._model(seq)
    prob = torch.nn.functional.softmax(predi, dim=0)
    deocde = tokenizer.decode(torch.argmax(predi, dim=0).item())
    print(f"Prédiction de la séquence {_seg} :  probabilité {prob.tolist()}, decode {deocde}")

print("porba si on avance")
show_proba_from_seq(['forward', 'empty', 'forward'])
show_proba_from_seq(['forward', 'wall', 'forward'])
show_proba_from_seq(['turn_left', 'empty', 'forward'])
show_proba_from_seq(['turn_right', 'empty', 'forward'])
show_proba_from_seq(['feel_front', 'empty', 'forward'])
show_proba_from_seq(['feel_front', 'wall', 'forward'])

print("porba si on turn left")
show_proba_from_seq(['forward', 'empty', 'turn_left'])
show_proba_from_seq(['forward', 'wall', 'turn_left'])
show_proba_from_seq(['turn_left', 'empty', 'turn_left'])
show_proba_from_seq(['turn_right', 'empty', 'turn_left'])
show_proba_from_seq(['feel_front', 'empty', 'turn_left'])
show_proba_from_seq(['feel_front', 'wall', 'turn_left'])

print("porba si on turn right")
show_proba_from_seq(['forward', 'empty', 'turn_right'])
show_proba_from_seq(['forward', 'wall', 'turn_right'])
show_proba_from_seq(['turn_left', 'empty', 'turn_right'])
show_proba_from_seq(['turn_right', 'empty', 'turn_right'])
show_proba_from_seq(['feel_front', 'empty', 'turn_right'])
show_proba_from_seq(['feel_front', 'wall', 'turn_right'])

print("porba si on feel front")
show_proba_from_seq(['forward', 'empty', 'feel_front'])
show_proba_from_seq(['forward', 'wall', 'feel_front'])
show_proba_from_seq(['turn_left', 'empty', 'feel_front'])
show_proba_from_seq(['turn_right', 'empty', 'feel_front'])
show_proba_from_seq(['feel_front', 'empty', 'feel_front'])
show_proba_from_seq(['feel_front', 'wall', 'feel_front'])

# Save in file txt agent._histoty_act and agent._history_fb

with open('history_agent.txt', 'w') as f:
    list_to_save = zip(agent_test2._history_act, agent_test2._history_fb)
    for x, y in list_to_save:
        f.write(str(f"{x}, {y}; ")) 
    f.write("\n")


# def show_decision_from_seq(_seg):
#     gap = (agent_test2._model.input_size - 1) // 2
#     x = []
#     for i in range(len(agent_test2._history_act) - gap, len(agent_test2._history_act)):
#         x.append(self._history_act[i])
#         x.append(agent_test2._history_fb[i])
#     seq = self._tokenizer.encode(x)
#     res = self.recursif_expective_valance(seq=seq, max_depth=3, seuil=0.2)
#     top_5 = sorted(res.items(), key=lambda x: x[1], reverse=True)[:5]
#     print(f"Top 5 of sequences with the best expected valance for {x}")
#     for top in top_5:
#         print(f"Sequence: {top[0]} Expected valance: {top[1]}")
    
#     print(f"Action choisie : {eval(top_5[0][0])[0]}")
#     return eval(top_5[0][0])[0]




test time training
multi op memory literal
